In [ ]:
import sklearn
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

In [ ]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define the parameter grid for GridSearchCV to find the best hyperparameters
param_grid = {
    'hidden_layer_sizes': [(10,), (30,), (50,), (10,10), (30,20)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [500, 1000]
}

In [ ]:
# Initialize MLPClassifier and perform GridSearchCV to find the best model
mlp = MLPClassifier(random_state=42)

grid_search = GridSearchCV(mlp, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score found by GridSearchCV
print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

In [ ]:
# Set the best parameters to the MLPClassifier and train the model
mlp.set_params(**grid_search.best_params_)
mlp.fit(X_train_scaled, y_train)

In [ ]:
# Function to calculate evaluation metrics (accuracy, precision, recall, F-score, specificity)
def evaluate_metrics(tn, fp, fn, tp):
  accuracy = (tp + tn) / (tp + tn + fp + fn)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f_score = 2 * (precision * recall) / (precision + recall)
  specificity = tn / (tn + fp)
  return accuracy, precision, recall, f_score, specificity

In [ ]:
# Predict on the test set and compute the confusion matrix
y_pred = mlp.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred)

# Initialize accumulators for metrics
accuracy_acm = 0
precision_acm = 0
recall_acm = 0
f_score_acm = 0
specificity_acm = 0

# Calculate and print metrics for each class
for i in range(len(iris.target_names)):
    tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
    fp = cm[:, i].sum() - cm[i, i]
    fn = cm[i, :].sum() - cm[i, i]
    tp = cm[i, i]

    accuracy, precision, recall, f_score, specificity = evaluate_metrics(tn, fp, fn, tp)

    accuracy_acm += accuracy
    precision_acm += precision
    recall_acm += recall
    f_score_acm += f_score
    specificity_acm += specificity

    print(f"Metrics for class {iris.target_names[i]}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F-score: {f_score}")
    print(f"Specificity: {specificity}")

In [ ]:
# Calculate and print overall metrics (averaged across all classes)
accuracy_overall = accuracy_acm / len(iris.target_names)
precision_overall = precision_acm / len(iris.target_names)
recall_overall = recall_acm / len(iris.target_names)
f_score_overall = f_score_acm / len(iris.target_names)
specificity_overall = specificity_acm / len(iris.target_names)

print("Metrics for the overall model:")
print(f"Average Accuracy: {accuracy_overall}")
print(f"Average Precision: {precision_overall}")
print(f"Average Recall: {recall_overall}")
print(f"Average F-score: {f_score_overall}")
print(f"Average Specificity: {specificity_overall}")

In [ ]:
# Binarize the labels for ROC curve calculation
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
y_score = mlp.predict_proba(X_test_scaled)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curves for each class
plt.figure()
colors = ['blue', 'red', 'green']
for i, color in zip(range(3), colors):
    plt.plot(fpr[i], tpr[i], color=color, label=f'ROC curve of class {i} (area = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC curve')
plt.legend()
plt.show()